In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
azure_openai_endpoint = os.environ["AZURE_OPENAI_ENDPOINT"]
azure_openai_key = os.environ["AZURE_OPENAI_KEY"]
neo4j_url = os.getenv("NEO4J_URI")
neo4j_username = os.getenv("NEO4J_USERNAME")
neo4j_password = os.getenv("NEO4J_PASSWORD")

In [4]:
from langchain_community.graphs import Neo4jGraph

graph = Neo4jGraph(database="relativity", url=neo4j_url, username=neo4j_username, password=neo4j_password)

In [ ]:
from langchain.document_loaders import WikipediaLoader

raw_documents = WikipediaLoader(query="General Relativity").load()

In [19]:
from langchain.text_splitter import TokenTextSplitter

text_splitter = TokenTextSplitter(chunk_size=512, chunk_overlap=24)
documents = text_splitter.split_documents(raw_documents[:3])

In [5]:
from langchain_openai import AzureChatOpenAI, AzureOpenAI

llm = AzureChatOpenAI (
    openai_api_version="2024-05-01-preview",
    api_key=azure_openai_key,
    azure_endpoint=azure_openai_endpoint,
    azure_deployment="gpt-35-turbo-16k",
    temperature=0.0,
)

llm2 = AzureOpenAI(
    openai_api_version="2024-05-01-preview",
    api_key=azure_openai_key,
    azure_endpoint=azure_openai_endpoint,
    azure_deployment="gpt-35-turbo-instruct",
    temperature=0.0
)

In [20]:
from langchain_experimental.graph_transformers import LLMGraphTransformer

llm_transformer = LLMGraphTransformer(llm=llm)

graph_documents = llm_transformer.convert_to_graph_documents(documents)
graph.add_graph_documents(
    graph_documents,
    baseEntityLabel=True,
    include_source=True
)

In [6]:
from langchain.chains import GraphCypherQAChain

graph.refresh_schema()
chain = GraphCypherQAChain.from_llm(
    llm, graph=graph, verbose=True,
)

In [8]:
chain.invoke({
    "query": "What is General Relativity related to?"
})



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (:Concept {id: "General Relativity"})-[:RELATED_TO]->(relatedConcept:Concept)
RETURN relatedConcept
Full Context:
[]

> Finished chain.


{'query': 'What is General Relativity related to?',
 'result': "I don't know the answer."}

In [9]:
from neo4j import GraphDatabase
from neo4j.graph import Node

def extract_graph_metadata(uri, username, password, database="neo4j"):
    driver = GraphDatabase.driver(uri, auth=(username, password), database=database)
    
    with driver.session() as session:
        node_details_query = "MATCH (n) RETURN n"
        node_details_result = session.run(node_details_query)
        node_details = [record["n"] for record in node_details_result]
        
        relationship_types_query = "CALL db.relationshipTypes()"
        relationship_types_result = session.run(relationship_types_query)
        relationship_types = [record["relationshipType"] for record in relationship_types_result]
        
        property_keys_query = "CALL db.propertyKeys()"     
        property_keys_result = session.run(property_keys_query)
        property_keys = [record["propertyKey"] for record in property_keys_result]
        
    return node_details, relationship_types, property_keys

uri = neo4j_url
username = neo4j_username
password = neo4j_password
database = "relativity"

node_details, relationship_types, property_keys = extract_graph_metadata(uri, username, password, database)

In [10]:
node_ids = [node['id'] for node in node_details if 'id' in node]
node_ids_str = ', '.join(node_ids)

relationship_types_str = ', '.join(relationship_types)

property_keys_str = ', '.join(property_keys)

In [11]:
query = "What is General Relativity related to?"

prompt = f"""
The graph contains {len(node_details)} nodes with the following IDs: {node_ids_str}.
The graph contains the following relationship types: {relationship_types_str}.
The graph contains the following property keys: {property_keys_str}.

Now answer the following question based on the information provided above:

{query}
"""

chain.invoke({
    'query': prompt,
})



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (:Concept {id: 'General_Relativity'})-[:RELATED_TO]->(c:Concept)
RETURN c.id
Full Context:
[{'c.id': 'Energy'}, {'c.id': 'Momentum'}, {'c.id': 'Matter'}, {'c.id': 'Radiation'}]

> Finished chain.


{'query': "\nThe graph contains 135 nodes with the following IDs: 52dd33d0f600f6c43a8dc97a76dd8a92, General_Relativity, General_Theory_Of_Relativity, Einstein_Theory_Of_Gravity, Albert_Einstein, Gravitation, Modern_Physics, Special_Relativity, Newton_Law_Of_Universal_Gravitation, Gravity, Geometric_Property_Of_Space_And_Time, Four_Dimensional_Spacetime, Curvature_Of_Spacetime, Energy, Momentum, Matter, Radiation, Einstein_Field_Equations, Second_Order_Partial_Differential_Equations, Classical_Gravity, Flat_Spacetime_Geometry, Stationary_Mass_Distributions, Predictions_Of_General_Relativity, Passage_Of_Time, Geometry_Of_Space, Motion_Of_Bodies_In_Free_Fall, Propagation_Of_Light, Gravitational_Time_Dilation, Gravitational_Lensing, Gravitational_Redshift_Of_Light, Shapiro_Time_Delay, Singularities, Black_Holes, Tests_Of_General_Relativity, Time-Dependent_Solutions_Of_General_Relativity, History_Of_The_Universe, Cosmology, Big_Bang, Cosmic_Microwave_Background_Radiation, Reconciliation_Of_

In [90]:
from yfiles_jupyter_graphs import GraphWidget
from neo4j import GraphDatabase

default_cypher = "MATCH (s)-[r:!MENTIONS]->(t) RETURN s,r,t LIMIT 10000"

def showGraph(cypher: str = default_cypher):
    # create a neo4j session to run queries
    driver = GraphDatabase.driver(
        # uri = os.environ["NEO4J_URI"],
        database="relativity",
        uri = "neo4j://localhost:7687",
        auth = (os.environ["NEO4J_USERNAME"],
                os.environ["NEO4J_PASSWORD"]))
    session = driver.session()
    widget = GraphWidget(graph = session.run(cypher).graph())
    widget.node_label_mapping = 'id'
    #display(widget)
    return widget

In [91]:
showGraph()

GraphWidget(layout=Layout(height='800px', width='100%'))